In [1]:
import polars as pl
import boto3
import xmltodict
import duckdb
import gzip
import json 
import copy
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import text, types
from sqlalchemy import Table, MetaData

import config

BUCKET = "data"

In [2]:
s3 = boto3.client('s3',                         
                  endpoint_url='http://localhost:9000',
                  aws_access_key_id='sOAg8CTUm62YCkj5mZLt',
                  aws_secret_access_key='h0Qm2r0gz7zc0Cp3gx4gNaKVndmkMvaBlZli7DMz'
)

def list_files(client, bucket: str):
    """List files in specific S3 URL"""
    response = client.list_objects(Bucket=bucket)
    for content in response.get('Contents', []):
        yield content.get('Key')

def parsing_infos_coll(dict_from_xml: dict):
    infos_dict = dict()
    dict_entete_doc = dict_from_xml["DocumentBudgetaire"]["EnTeteDocBudgetaire"]
    infos_dict["siret_coll"] = dict_entete_doc["IdColl"]["@V"]
    infos_dict["libelle_collectivite"] = dict_entete_doc["LibelleColl"]["@V"]
    infos_dict["nature_collectivite"] = dict_entete_doc["NatCEPL"]["@V"]
    infos_dict["departement"] = dict_entete_doc.get("Departement", {}).get("@V", None)

    return infos_dict

def parsing_infos_etablissement(dict_from_xml: dict, id_doc_budg):
    infos_dict = dict()
    dict_entete_budget = dict_from_xml["DocumentBudgetaire"]["Budget"]["EnTeteBudget"]
    dict_bloc_budget = dict_from_xml["DocumentBudgetaire"]["Budget"]["BlocBudget"]

    infos_dict["id_doc_budg"] = id_doc_budg
    
    #Parsing entête budgétaire
    infos_dict["siret_etablissement"] = dict_entete_budget["IdEtab"]["@V"]
    infos_dict["libelle"] = dict_entete_budget["LibelleEtab"]["@V"]
    infos_dict["code_insee"] = dict_entete_budget.get(
        "CodInseeColl", {}).get("@V", None)
    infos_dict["nomenclature"] = dict_entete_budget["Nomenclature"]["@V"]
    
    # Parsing bloc budget
    infos_dict["exercice"] = int(dict_bloc_budget["Exer"]["@V"])
    infos_dict["nature_dec"] = dict_bloc_budget["NatDec"]["@V"]
    infos_dict["num_dec"] = int(dict_bloc_budget.get(
        "NumDec", {}).get("@V", None) or 0)
    infos_dict["nature_vote"] = dict_bloc_budget["NatFonc"]["@V"]
    infos_dict["type_budget"] = dict_bloc_budget["CodTypBud"]["@V"]
    infos_dict["id_etabl_princ"] = dict_bloc_budget.get(
        "IdEtabPal", {}).get("@V", None)

    # Parsing des lignes de budget -> json
    infos_dict["json_budget"] = generate_dict_budget(dict_from_xml)
    # duplique certaines données et non nécessaire pour l'instant donc je le commente
    #if isinstance(dict_from_xml["DocumentBudgetaire"]["Budget"]["Annexes"], dict):
    #    infos_dict["list_annexes"] = list(dict_from_xml["DocumentBudgetaire"]["Budget"]["Annexes"].keys())
    
    infos_dict["fk_siret_collectivite"] = dict_from_xml["DocumentBudgetaire"]["EnTeteDocBudgetaire"]["IdColl"]["@V"]

    return infos_dict

def generate_dict_budget(dict_from_xml: dict) -> dict:
        budget_dict = copy.deepcopy(
            dict_from_xml["DocumentBudgetaire"]["Budget"]["LigneBudget"])
        
        if isinstance(budget_dict, dict):
            for field in config.CHAMPS_LIGNE_BUDGET:
                if field in budget_dict:
                    if "@V" in budget_dict[field]:
                        budget_dict[field] = budget_dict[field]['@V']
            # for field in ["MtSup", "CaracSup"]:
            #     if field in budget_dict:
            #         if "@V" in budget_dict[field]:
            #             budget_dict[field] = { budget_dict[field]['@Code']: budget_dict[field]['@V'] }
            return budget_dict
        for idx, row in enumerate(budget_dict):
            for field in config.CHAMPS_LIGNE_BUDGET:
                if field in row:
                    if "@V" in row[field]:
                        budget_dict[idx][field] = row[field]['@V']
            for field in ["MtSup", "CaracSup"]:
                if field in row:
                    budget_dict[idx].pop(field)
            # for field in ["MtSup", "CaracSup"]:
            # if field in row:
            #     if "@V" in row[field]:
            #         budget_dict[idx][field] = { row[field]['@Code']: row[field]['@V'] }
        return budget_dict

In [15]:
champs_budg = config.CHAMPS_LIGNE_BUDGET

def _all_annexe_columns(df, fields):
    all_columns = copy.deepcopy(fields)
    columns = list(df.columns)
    for i in all_columns:
        columns.append(i)
    return columns

def explode_annexe_json_into_rows_first_way(df, fields):
    all_columns = _all_annexe_columns(df, fields)
    all_columns.pop(all_columns.index("json_budget"))
    df['json_budget'] = df.json_budget.apply(lambda x:eval(str(x)))
    temp = df.groupby('id_doc_budg').json_budget.apply(lambda x: pd.DataFrame(x.values[0], index=[0]).reset_index())
    df.drop(columns="json_budget", inplace=True)
    df_result = df.merge(temp, left_on='id_doc_budg', right_on='id_doc_budg')
    return df_result.reindex(columns = all_columns)

def explode_annexe_json_into_rows_second_way(df, fields):
    all_columns = _all_annexe_columns(df, fields)
    s= df.set_index('id_doc_budg').json_budget.apply(lambda x:eval(str(x))).explode()
    temp = pd.DataFrame(s.tolist(), index = s.index).reset_index()
    df.drop(columns="json_budget", inplace=True)
    df_result = temp.merge(df, left_on='id_doc_budg', right_on='id_doc_budg')
    return df_result.reindex(columns = all_columns)

In [282]:
count = 0
print(len(list(list_files(s3, BUCKET))))
for file in list_files(s3, BUCKET):
    id_doc_budg = file.split("-")[1].split(".")[0]

    response = s3.get_object(Bucket=BUCKET,
                        Key=file)
    dict_from_xml = xmltodict.parse(
            gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
        )
    temp_df = pd.DataFrame.from_dict(
            parsing_infos_etablissement(dict_from_xml, id_doc_budg),
        )
    
    if count == 0:
        all_columns = _all_annexe_columns(temp_df, config.CHAMPS_LIGNE_BUDGET)
        #all_columns.pop(all_columns.index("json_budget"))
        schema = pd.DataFrame(columns=all_columns)
        engine = create_engine('sqlite:///ab_budget.db', echo=False)
        schema.to_sql('ab_budg', con=engine, if_exists="replace",
                      dtype={
                    "id_doc_budg": types.Integer,
                    "siret_etablissement": types.Text,
                    "libelle": types.Text,
                    "code_insee": types.Text,
                    "nomenclature": types.Text,
                    "exercice": types.Integer,
                    "nature_dec": types.Text,
                    "num_dec": types.Integer,
                    "nature_vote": types.Text,
                    "type_budget": types.Text,
                    "id_etabl_princ": types.Text,
                    "fk_siret_collectivite": types.Text,
                    "Nature": types.Text,
                    "LibCpte": types.Text,
                    "Fonction": types.Text,
                    "Operation": types.Text,
                    "ContNat": types.Text,
                    "ArtSpe": types.Text,
                    "ContFon": types.Text,
                    "ContOp": types.Text,
                    "CodRD": types.Text,
                    "MtBudgPrec": types.Float,
                    "MtRARPrec": types.Float,
                    "MtPropNouv": types.Text,
                    "MtPrev": types.Text,
                    "CredOuv": types.Text,
                    "MtReal": types.Float,
                    "MtRAR3112": types.Float,
                    "OpBudg": types.Text,
                    "TypOpBudg": types.Text,
                    "OpeCpteTiers": types.Text
                    }
                    )
        count = 1

    temp_df = explode_annexe_json_into_rows_first_way(temp_df, config.CHAMPS_LIGNE_BUDGET)

    temp_df.to_sql('ab_budg', con=engine, if_exists='append', index=False)

1000


In [23]:
count = 0
print(len(list(list_files(s3, BUCKET))))
for file in list_files(s3, BUCKET):
    id_doc_budg = file.split("-")[1].split(".")[0]

    response = s3.get_object(Bucket=BUCKET,
                        Key=file)
    dict_from_xml = xmltodict.parse(
            gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
        )
    temp_df = pd.DataFrame.from_dict(
            parsing_infos_etablissement(dict_from_xml, id_doc_budg),
        )
    
    if count == 0:
        conn = duckdb.connect('actes_budgetaires.duckdb')
        conn.execute("CREATE OR REPLACE TABLE budget ( \
                     id_doc_budg UINTEGER, \
                    siret_etablissement VARCHAR, \
                    libelle VARCHAR, \
                    code_insee VARCHAR, \
                    nomenclature VARCHAR, \
                    exercice UINTEGER, \
                    nature_dec VARCHAR, \
                    num_dec UINTEGER, \
                    nature_vote VARCHAR, \
                    type_budget VARCHAR, \
                    id_etabl_princ VARCHAR, \
                    fk_siret_collectivite VARCHAR, \
                    Nature VARCHAR, \
                    LibCpte VARCHAR, \
                    Fonction VARCHAR, \
                    Operation VARCHAR, \
                    ContNat VARCHAR, \
                    ArtSpe VARCHAR, \
                    ContFon VARCHAR, \
                    ContOp VARCHAR, \
                    CodRD VARCHAR, \
                    MtBudgPrec DOUBLE, \
                    MtRARPrec DOUBLE, \
                    MtPropNouv VARCHAR, \
                    MtPrev VARCHAR, \
                    CredOuv VARCHAR, \
                    MtReal DOUBLE, \
                    MtRAR3112 DOUBLE, \
                    OpBudg VARCHAR, \
                    TypOpBudg VARCHAR, \
                    OpeCpteTiers VARCHAR)")
        count = 1

    temp_df = explode_annexe_json_into_rows_first_way(temp_df, config.CHAMPS_LIGNE_BUDGET)
    conn.sql("INSERT INTO budget SELECT * FROM temp_df")


866


In [28]:
conn.sql("SELECT * FROM budget")


┌─────────────┬─────────────────────┬──────────────────────┬───┬───────────┬─────────┬───────────┬──────────────┐
│ id_doc_budg │ siret_etablissement │       libelle        │ … │ MtRAR3112 │ OpBudg  │ TypOpBudg │ OpeCpteTiers │
│   uint32    │       varchar       │       varchar        │   │  double   │ varchar │  varchar  │   varchar    │
├─────────────┼─────────────────────┼──────────────────────┼───┼───────────┼─────────┼───────────┼──────────────┤
│     1810875 │ 20009162700022      │ REGIE SPANC          │ … │       0.0 │ 0       │ NULL      │ NULL         │
│     1810875 │ 20009162700022      │ REGIE SPANC          │ … │       0.0 │ 0       │ NULL      │ NULL         │
│     1810875 │ 20009162700022      │ REGIE SPANC          │ … │       0.0 │ 0       │ NULL      │ NULL         │
│     1810875 │ 20009162700022      │ REGIE SPANC          │ … │       0.0 │ 0       │ NULL      │ NULL         │
│     1810875 │ 20009162700022      │ REGIE SPANC          │ … │       0.0 │ 0       │ N

In [275]:
with engine.connect() as conn:
   result = conn.execute(text("SELECT * FROM ab_budg")).fetchall()


In [276]:
df_budg = pd.read_sql('SELECT * FROM ab_budg', 'sqlite:///ab_budget.db', index_col="index")

In [277]:
pd.set_option("display.max.columns", None)
df_budg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119307 entries, 0 to 0
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id_doc_budg            119307 non-null  int64  
 1   siret_etablissement    119307 non-null  object 
 2   libelle                119307 non-null  object 
 3   code_insee             88133 non-null   object 
 4   nomenclature           119307 non-null  object 
 5   exercice               119307 non-null  int64  
 6   nature_dec             119307 non-null  object 
 7   num_dec                119307 non-null  int64  
 8   nature_vote            119307 non-null  object 
 9   type_budget            119307 non-null  object 
 10  id_etabl_princ         21895 non-null   object 
 11  fk_siret_collectivite  119307 non-null  object 
 12  Nature                 119307 non-null  object 
 13  LibCpte                24409 non-null   object 
 14  Fonction               67808 non-null   object

In [177]:
file = list(list_files(s3, bucket))[5]

print(file)
id_doc_budg = file.split("-")[1].split(".")[0]

response = s3.get_object(Bucket=bucket,
                    Key=file)
dict_from_xml = xmltodict.parse(
        gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
    )
temp_df = pl.from_dicts(
        parsing_infos_etablissement(dict_from_xml, id_doc_budg),
    schema_overrides={"json_budget pl.Struct},
    infer_schema_length=None
    ).unnest("json_budget")

duckdb.sql('SELECT * FROM temp_df').show()

20200801-1810890.xml.gz
┌─────────────┬─────────────────────┬──────────────────┬───┬─────────┬───────────┬───────────┬──────────────────────┐
│ id_doc_budg │ siret_etablissement │     libelle      │ … │ OpBudg  │ TypOpBudg │ MtRARPrec │ fk_siret_collectiv…  │
│   varchar   │       varchar       │     varchar      │   │ varchar │  varchar  │  varchar  │       varchar        │
├─────────────┼─────────────────────┼──────────────────┼───┼─────────┼───────────┼───────────┼──────────────────────┤
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019  

In [191]:

count = 0
file = "20200801-1810947.xml.gz"
    
print(file)
id_doc_budg = file.split("-")[1].split(".")[0]

response = s3.get_object(Bucket=bucket,
                    Key=file)
dict_from_xml = xmltodict.parse(
        gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
    )
temp_df = pd.DataFrame.from_dict(
        parsing_infos_etablissement(dict_from_xml, id_doc_budg),
    )
    
all_columns = _all_annexe_columns(temp_df, config.CHAMPS_LIGNE_BUDGET)
temp_df['json_budget'] = temp_df.json_budget.apply(lambda x:eval(str(x)))
temp = temp_df.groupby('id_doc_budg').json_budget.apply(lambda x: pd.DataFrame(x.values[0], index=[0]).reset_index())
temp_df.drop(columns="json_budget", inplace=True)
df_result = temp_df.merge(temp, left_on='id_doc_budg', right_on='id_doc_budg')
df_result.reindex(columns = all_columns)


20200801-1810947.xml.gz


In [218]:
parsing_infos_etablissement({}, id_doc_budg)
"siret_etablissement", "libelle", "code_insee", "nomenclature", 

KeyError: 'DocumentBudgetaire'

In [261]:
temp_df.dtypes

id_doc_budg              object
siret_etablissement      object
libelle                  object
code_insee               object
nomenclature             object
exercice                  int64
nature_dec               object
num_dec                   int64
nature_vote              object
type_budget              object
id_etabl_princ           object
fk_siret_collectivite    object
index                     int64
Nature                   object
LibCpte                  object
ContNat                  object
ArtSpe                   object
CodRD                    object
MtBudgPrec               object
MtRARPrec                object
MtPropNouv               object
MtPrev                   object
CredOuv                  object
MtReal                   object
MtRAR3112                object
OpBudg                   object
dtype: object